In [1]:
%matplotlib qt

import time
import numpy as np
from skimage.filters import gaussian_filter

from tifffile import imread, imsave
import cPickle as pickle

from spimagine import volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

from netsurface3d import NetSurf3d

<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


/Users/jug/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pyopencl/__init__.py:207: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


could not open /Users/jug/.spimagine
<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


# Load dataset and show in Spimagine 

In [8]:
filename = '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t1.TIF'
image = imread(filename)
print 'Image dimensions: ', image.shape

volwin = volshow(image, stackUnits = [1.,1.,5.], raise_window=False)
print 'Dataset now visible in separate Spimagine window...'

Image dimensions:  (31, 512, 512)
Dataset now visible in separate Spimagine window...


# Load vectors sampling unit sphere 

In [3]:
# load pickeled unit sphere sampling
with open('sphere_sampling.pkl','r') as f:
    dictSphereData = pickle.load(f)
    
# sampling parameters
vectors = dictSphereData['points']
neighbors = dictSphereData['neighbors']
neighbors_of = dictSphereData['neighbors_of']
triangles = dictSphereData['triangles']

# Do segmentations using NetSurf3d

In [4]:
K = 30
max_delta_k = 3

# E cells
# -------
cell1 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )
cell1_center = (110,300,12)

cell2 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )
cell2_center = (110,350,12)

# Neighbors
# ---------
nn = 11
cells = [None] * nn
ccenters = [None] * nn

max_delta_ks = [3] * nn
max_delta_ks[0] = 2
max_delta_ks[7] = 2

for i in range(nn):
    cells[i] = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_ks[i] )

ccenters[0]  = ( 90,390,12)
ccenters[1]  = (150,410,12)
ccenters[2]  = (200,390,12)
ccenters[3]  = (190,315,12)
ccenters[4]  = (145,265,12)
ccenters[5]  = (110,230,12)
ccenters[6]  = ( 40,255,12)
ccenters[7]  = ( 45,300,18)
ccenters[8]  = ( 67,355,18)

ccenters[9]  = (110,300,22)
ccenters[10] = (110,350,22)

# ccenters[11] = ( 70,200,12)


In [5]:
max_rs = (image.shape[2] / 6,
          image.shape[1] / 6,
          image.shape[0] / 3)

min_rs = (10,10,2)

optimum = cell1.apply_to( image, cell1_center, max_rs, min_radii=min_rs )
print 'Optimum energy cell 1: ', optimum
ins, outs = cell1.get_counts()
print 'Nodes in/out: ', ins, outs

optimum = cell2.apply_to( image, cell2_center, max_rs ) #, min_radii=min_rs )
print 'Optimum energy cell 2: ', optimum
ins, outs = cell2.get_counts()
print 'Nodes in/out: ', ins, outs

print 'Segmenting ', nn, ' more cells: ',
for i in range(nn):
    cells[i].apply_to( image, ccenters[i], max_rs, min_radii=min_rs )
    print '.',
print ' done!'

Optimum energy cell 1:  112544.0
Nodes in/out:  5509 9851
Optimum energy cell 2:  94873.0
Nodes in/out:  6771 8589
Segmenting  11  more cells:  . . . . . . . . . . .  done!


In [6]:
colors_red = [(1.,.2,.2*i) for i in range(6)]
colors_gold = [(1.,.8,.15*i) for i in range(6)]
colors_yellow = [(1.,1.,.9-.15*i) for i in range(6)]
colors_green = [(.4,1.,.9-.1*i) for i in range(6)]
colors_blue = [(0.,0.,.9-.15*i) for i in range(6)]

colors = colors_green

In [17]:
#remove all meshes
while len(volwin.glWidget.meshes)>0:
    volwin.glWidget.meshes.pop(0)
volwin.glWidget.refresh()

In [18]:
mesh1 = cell1.create_surface_mesh( facecolor=colors_gold[0] )
volwin.glWidget.add_mesh( mesh1 )

mesh2 = cell2.create_surface_mesh( facecolor=colors_gold[2] )
volwin.glWidget.add_mesh( mesh2 )

In [19]:
for i in range(nn):
    fc = colors[i%len(colors)]
    m = cells[i].create_center_mesh( facecolor=fc )
    volwin.glWidget.add_mesh( m )

In [20]:
for i in range(nn-2):
    fc = colors[i%len(colors)]
    m = cells[i].create_surface_mesh( facecolor=fc )
    volwin.glWidget.add_mesh( m )

# Measure Volume

In [78]:
calibration = (1.,1.,5.)

v1 = cell1.get_volume(calibration)
v2 = cell2.get_volume(calibration)
print 'Volume of e cell 1:', v1, ' - thats equivalent to a cube of side length ', v1**(1./3)
print 'Volume of e cell 2:', v2, ' - thats equivalent to a cube of side length ', v2**(1./3)
print 

for i in range(nn):
    v = cells[i].get_volume(calibration)
    print 'Volume of cell',i+1,' is ', v

#idea 2: filter all points between to results that are closer then come value d. compute area of all 
#        triangles defines by those points - touching surface!

Volume of e cell 1: 135744.166667  - thats equivalent to a cube of side length  51.3933655165
Volume of e cell 2: 124986.666667  - thats equivalent to a cube of side length  49.998222159

Volume of cell 1  is  50896.6666667
Volume of cell 2  is  56050.0
Volume of cell 3  is  127605.833333
Volume of cell 4  is  113885.0
Volume of cell 5  is  86240.0
Volume of cell 6  is  146724.166667
Volume of cell 7  is  100952.5
Volume of cell 8  is  142212.5
